In [ ]:
import sys
sys.path.append('../src')
from train_process.train_CNN import train


In [ ]:
train_args = {
    "eeg_file_names": ["1","16"],  # 示例文件名
    "test_mode": False,
    "image_size": 32,
    "batch_size": 64,
    "train_size": 0.7,
    "val_size": 0.2,
    "test_size": 0.1,
    "out": "D:/Research/EEG/EEG_DL_Classifier/checkpoints",
    "wandb_run_name": "EEG_Classification_Run",
    "disable_wandb": False,
    "enable_progress_bar": True,
    "max_epochs": 30  # 示例训练周期数
}

train(train_args)
